In [2]:
import sys
sys.path.append("../..") 

from custom_scripts import database
from custom_scripts import preprocessing

In [3]:
### Table of unique carrier identifiers present in the test samples
carriers = database.query(""" 
                            SELECT op_unique_carrier, COUNT(*)
                                FROM flights_test
                                WHERE fl_date = ANY('{2020-01-01, 2020-01-02, 2020-01-03, 2020-01-04, 2020-01-05, 2020-01-06, 2020-01-07}')
                                GROUP BY op_unique_carrier;
                            """)
carriers.to_csv('../../data/preprocessing/unique_carriers.csv', index=False)
carriers.head(1)

,op_unique_carrier,count
0,9E,4942


In [4]:
### Table of flight numbers present in the test samples, and a count of their frequency
test_flight_numbers = database.query("""
                                SELECT op_carrier_fl_num, COUNT(*) 
                                    FROM flights_test
                                    WHERE fl_date = ANY('{{2020-01-31, 2020-01-30, 2020-01-29, 2020-01-28, 2020-01-27, 2020-01-26, 2020-01-25}}')
                                    GROUP BY op_carrier_fl_num;
                                """)
test_flight_numbers.to_csv('../../data/preprocessing/test_flight_numbers.csv', index=False)
test_flight_numbers.head(1)

,op_carrier_fl_num,count
0,6114,14


In [14]:
def create_average_table(feature,prefix):
    table = database.query(f"""
                            SELECT  {feature},
                                    AVG(dep_delay) AS "{prefix}_avg_dep_delay", 
                                    AVG(taxi_out) AS "{prefix}_taxi_out",
                                    AVG(wheels_off) AS "{prefix}_wheels_off", 
                                    AVG(wheels_on) AS "{prefix}_wheels_on", 
                                    AVG(taxi_in) AS "{prefix}_taxi_in", 
                                    AVG(arr_delay) AS "{prefix}_avg_arr_delay",
                                    AVG(crs_elapsed_time) AS "{prefix}_crs_elapsed_time",
                                    AVG(actual_elapsed_time) AS "{prefix}_actual_elapsed_time",
                                    AVG(air_time) AS "{prefix}_air_time",
                                    AVG(carrier_delay) AS "{prefix}_carrier_delay",
                                    AVG(weather_delay) AS "{prefix}_weather_delay",
                                    AVG(nas_delay) AS "{prefix}_avg_nas_delay", 
                                    AVG(security_delay) AS "{prefix}_security_delay",
                                    AVG(late_aircraft_delay) AS "{prefix}_late_aircraft_delay",
                                    AVG(total_add_gtime) AS "{prefix}_total_add_gtime",
                                    AVG(longest_add_gtime) AS "{prefix}_longest_add_gtime"                                      
                                FROM flights
                                GROUP BY {feature};
                            """)
    table.to_csv(f'../../data/preprocessing/averages_by_{prefix}.csv', index=False)
    return table

In [16]:
table = preprocessing.create_average_table('op_carrier_fl_num','fl_num')
table.head(1)

,op_carrier_fl_num,fl_num_avg_dep_delay,fl_num_taxi_out,fl_num_wheels_off,fl_num_wheels_on,fl_num_taxi_in,fl_num_avg_arr_delay,fl_num_crs_elapsed_time,fl_num_actual_elapsed_time,fl_num_air_time,fl_num_carrier_delay,fl_num_weather_delay,fl_num_avg_nas_delay,fl_num_security_delay,fl_num_late_aircraft_delay,fl_num_total_add_gtime,fl_num_longest_add_gtime
0,1,2.7734,16.922582,858.495249,1063.479598,8.920347,-5.716606,262.095996,256.171661,230.33492,24.254237,1.391949,12.355932,0.254237,6.618644,22.192308,22.192308


In [7]:
### Historic average delay times by flight number
avg_flight_delays_by_flight_number = database.query("""
                                SELECT  op_carrier_fl_num,
                                        AVG(dep_delay) AS "fl_num_avg_dep_delay", 
                                        AVG(taxi_out) AS "fl_num_taxi_out",
                                        AVG(wheels_off) AS "fl_num_wheels_off", 
                                        AVG(wheels_on) AS "fl_num_wheels_on", 
                                        AVG(taxi_in) AS "fl_num_taxi_in", 
                                        AVG(arr_delay) AS "fl_num_avg_arr_delay",
                                        AVG(crs_elapsed_time) AS "fl_num_crs_elapsed_time",
                                        AVG(actual_elapsed_time) AS "fl_num_actual_elapsed_time",
                                        AVG(air_time) AS "fl_num_air_time",
                                        AVG(carrier_delay) AS "fl_num_carrier_delay",
                                        AVG(weather_delay) AS "fl_num_weather_delay",
                                        AVG(nas_delay) AS "fl_num_avg_nas_delay", 
                                        AVG(security_delay) AS "fl_num_security_delay",
                                        AVG(late_aircraft_delay) AS "fl_num_late_aircraft_delay",
                                        AVG(total_add_gtime) AS "fl_num_total_add_gtime",
                                        AVG(longest_add_gtime) AS "fl_num_longest_add_gtime"                                      
                                    FROM flights
                                    GROUP BY op_carrier_fl_num;
                                """)
avg_flight_delays_by_flight_number.to_csv('../../data/preprocessing/average_delays_by_flight_number.csv', index=False)
avg_flight_delays_by_flight_number.head(1)


,op_carrier_fl_num,fl_num_avg_dep_delay,fl_num_taxi_out,fl_num_wheels_off,fl_num_wheels_on,fl_num_taxi_in,fl_num_avg_arr_delay,fl_num_crs_elapsed_time,fl_num_actual_elapsed_time,fl_num_air_time,fl_num_carrier_delay,fl_num_weather_delay,fl_num_avg_nas_delay,fl_num_security_delay,fl_num_late_aircraft_delay,fl_num_total_add_gtime,fl_num_longest_add_gtime
0,1,2.7734,16.922582,858.495249,1063.479598,8.920347,-5.716606,262.095996,256.171661,230.33492,24.254237,1.391949,12.355932,0.254237,6.618644,22.192308,22.192308


In [5]:
### Average flight delays grouped by destination airport ID
# avg_delays_by_destination_airport = database.query("""
#                                 SELECT  dest_airport_id, 
#                                         AVG(arr_delay) AS "dest_avg_arr_delay",
#                                         AVG(dep_delay) AS "dest_airport_id_avg_dep_delay", 
#                                         AVG(taxi_out) AS "dest_airport_id_taxi_out",
#                                         AVG(wheels_off) AS "dest_airport_id_wheels_off", 
#                                         AVG(wheels_on) AS "dest_airport_id_wheels_on", 
#                                         AVG(taxi_in) AS "dest_airport_id_taxi_in", 
#                                         AVG(arr_delay) AS "dest_airport_id_avg_arr_delay",
#                                         AVG(crs_elapsed_time) AS "dest_airport_id_crs_elapsed_time",
#                                         AVG(actual_elapsed_time) AS "dest_airport_id_actual_elapsed_time",
#                                         AVG(air_time) AS "dest_airport_id_air_time",
#                                         AVG(carrier_delay) AS "dest_airport_id_carrier_delay",
#                                         AVG(weather_delay) AS "dest_airport_id_weather_delay",
#                                         AVG(nas_delay) AS "dest_airport_id_avg_nas_delay", 
#                                         AVG(security_delay) AS "dest_airport_id_security_delay",
#                                         AVG(late_aircraft_delay) AS "dest_airport_id_late_aircraft_delay",
#                                         AVG(total_add_gtime) AS "dest_airport_id_total_add_gtime",
#                                         AVG(longest_add_gtime) AS "dest_airport_id_longest_add_gtime"      
#                                     FROM flights
#                                     GROUP BY dest_airport_id;
#                                 """)
# avg_delays_by_destination_airport.to_csv('../../data/preprocessing/avg_delays_by_destination_airport.csv', index=False)
# avg_delays_by_destination_airport.head(1)

SyntaxError: syntax error at or near "FROM"
LINE 9:                                     FROM flights
                                            ^


In [9]:
### Average flight delays grouped by origin airport ID
# avg_flight_delays_by_origin_airport = database.query("""
#                                 SELECT  origin_airport_id, 
#                                         AVG(arr_delay) AS "origin_avg_arr_delay",
#                                         AVG(carrier_delay) AS "origin_avg_carrier_delay", 
#                                         AVG(weather_delay) AS "origin_id_avg_weather_delay", 
#                                         AVG(nas_delay) AS "origin_avg_nas_delay", 
#                                         AVG(late_aircraft_delay) AS "origin_avg_late_aircraft_delay",
#                                         AVG(dep_delay) AS "origin_avg_dep_delay"
#                                     FROM flights
#                                     GROUP BY origin_airport_id;
#                                 """)
# avg_flight_delays_by_origin_airport.to_csv('../../data/preprocessing/avg_flight_delays_by_origin_airport.csv', index=False)
# avg_flight_delays_by_origin_airport.head(1)

,origin_airport_id,avg_arr_delay,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_late_aircraft_delay,avg_dep_delay
0,10135,7.024949,24.29115,4.138496,17.511504,33.039823,10.58383


In [3]:
### Average flight delays grouped by carrier ID
# avg_flight_delays_by_carrier = database.query("""
#                                 SELECT  op_unique_carrier, 
#                                         AVG(arr_delay) AS "carrier_avg_arr_delay",
#                                         AVG(carrier_delay) AS "carrier_avg_carrier_delay", 
#                                         AVG(weather_delay) AS "carrier_avg_weather_delay", 
#                                         AVG(nas_delay) AS "carrier_avg_nas_delay", 
#                                         AVG(late_aircraft_delay) AS "carrier_avg_late_aircraft_delay",
#                                         AVG(dep_delay) AS "avg_dep_delay"
#                                     FROM flights
#                                     GROUP BY op_unique_carrier;
#                                 """)
# avg_flight_delays_by_carrier.to_csv('../../data/preprocessing/avg_flight_delays_by_carrier.csv', index=False)
# avg_flight_delays_by_carrier.head(1)

,op_unique_carrier,avg_arr_delay,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_late_aircraft_delay,avg_dep_delay
0,9E,3.788254,20.54492,5.287241,21.227495,31.674297,10.550252


In [3]:
avg_flight_delays_by_day_of_year = database.query("""
                                  SELECT  extract(doy from TO_DATE(fl_date,'YYYY-MM-DD')) AS "day_of_year", 
                                        AVG(arr_delay) AS "day_of_year_avg_arr_delay",
                                        AVG(carrier_delay) AS "day_of_year_avg_carrier_delay", 
                                        AVG(weather_delay) AS "day_of_year_avg_weather_delay", 
                                        AVG(nas_delay) AS "day_of_year_avg_nas_delay", 
                                        AVG(late_aircraft_delay) AS "day_of_year_avg_late_aircraft_delay",
                                        AVG(dep_delay) AS "day_of_year_avg_dep_delay"
                                    FROM flights
                                    GROUP BY day_of_year;
                                """)
avg_flight_delays_by_day_of_year.to_csv('../../data/preprocessing/avg_flight_delays_by_day_of_year.csv', index=False)
avg_flight_delays_by_day_of_year.head(1)

,day_of_year,avg_arr_delay,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_late_aircraft_delay,avg_dep_delay
0,1.0,10.488622,24.226509,3.270192,9.308377,28.867853,16.427019


In [4]:
avg_flight_delays_by_day_of_year


,day_of_year,avg_arr_delay,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_late_aircraft_delay,avg_dep_delay
0,1.0,10.488622,24.226509,3.270192,9.308377,28.867853,16.427019
1,2.0,9.968470,20.181292,3.097133,9.627948,27.497326,15.290052
2,3.0,4.861474,21.895186,2.675742,8.769455,24.413181,11.164983
3,4.0,-0.225516,23.994110,5.239463,8.960795,22.033867,7.009716
4,5.0,7.331032,25.348407,4.181409,17.958879,29.420204,13.643319
...,...,...,...,...,...,...,...
360,361.0,10.692640,20.433123,4.743057,10.104277,30.129068,16.406279
361,362.0,16.816636,20.640679,3.894288,13.002324,31.244876,21.610000
362,363.0,9.349526,22.756895,3.178721,8.778265,28.472237,15.625149
363,364.0,7.604898,20.687131,3.865298,12.472308,28.440461,13.428252
